## Styles & Dependencies

In [1]:
# Main Dependencies
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# Matplotlib Dependencies & Setup
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from pandas.plotting import table

## SQL Query Engine and ORM Setup

In [3]:
# SQL Alchemy dependencies (Python SQL toolkit and ORM)
import sqlalchemy
from sqlalchemy import extract
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
# Create our session (link) from Python to the DB
session = Session(engine)

##  Query #1: Determine the Summary Statistics for June

In [6]:
# Query: filter Measurement table to retrieve temperatures for June
june_temps = session.query(Measurement).filter(extract('month', Measurement.date) == 6)

In [7]:
# Convert June temperatures to a list
june_temps_list = [temp.tobs for temp in june_temps]
#print(june_temps_list)

In [37]:
# Get summary stats from june_temps_list (to compare with DF stats)
print(np.mean(june_temps_list))
print(np.median(june_temps_list))
print(np.min(june_temps_list))
print(np.max(june_temps_list))
print(np.std(june_temps_list))

74.94411764705882
75.0
64.0
85.0
3.2564588468846476


In [9]:
# Create a DataFrame from the list of temperatures for the month of June. 
june_temps_df = pd.DataFrame(june_temps_list, columns=["temps"])

In [10]:
# Calculate and print out the summary statistics for the June temperature DataFrame.
june_temps_df.describe()

,temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## Query #2: Determine the Summary Statistics for December

In [11]:
# Query: filter Measurement table to retrieve temperatures for December
december_temps = session.query(Measurement).filter(extract('month', Measurement.date) == 12)

In [12]:
# Convert the December temperatures to a list.
december_temps_list = [temp.tobs for temp in december_temps]
#print(december_temps_list)

In [38]:
# Get summary stats from december_temps_list (to compare with DF stats)
print(np.mean(december_temps_list))
print(np.median(december_temps_list))
print(np.min(december_temps_list))
print(np.max(december_temps_list))
print(np.std(december_temps_list))

71.04152933421226
71.0
56.0
83.0
3.744684974507512


In [14]:
# Create a DataFrame from the list of temperatures for the month of December. 
december_temps_df = pd.DataFrame(december_temps_list, columns=["temps"])

In [15]:
# Calculate and print out the summary statistics for the December temperature DataFrame.
december_temps_df.describe()

,temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


## Exploratory Analysis

In [ ]:
# Query hawaii weather db to extract all colums for exploratory analysis
#results = session.query(Measurement.date, Measurement.).all()

## Query #3: Precipitation Analysis for June

In [26]:
# Query: filter Measurement table to retrieve precipitation data for June
june_prcp = session.query(Measurement.prcp).filter(extract('month', Measurement.date) == 6)
print(june_prcp)

SELECT measurement.prcp AS measurement_prcp 
FROM measurement 
WHERE CAST(STRFTIME('%m', measurement.date) AS INTEGER) = ?


In [28]:
# Convert the June precipitation data from table to a list.
june_prcp_list = [prcp.prcp for prcp in june_prcp]
#print(june_prcp_list)

In [30]:
# Create a DataFrame from June_prcp_list
june_prcp_df = pd.DataFrame(june_prcp_list, columns=["prcp"])

In [31]:
# Use Pandas to calcualte the summary statistics for the precipitation data
june_prcp_df.describe()

,prcp
count,1574.000000
mean,0.136360
std,0.335731
min,0.000000
25%,0.000000
50%,0.020000
75%,0.120000
max,4.430000


## Query #4: Precipitation Analysis for December

In [32]:
# Query: filter Measurement table to retrieve precipitation data for December
dec_prcp = session.query(Measurement.prcp).filter(extract('month', Measurement.date) == 12)
print(dec_prcp)

SELECT measurement.prcp AS measurement_prcp 
FROM measurement 
WHERE CAST(STRFTIME('%m', measurement.date) AS INTEGER) = ?


In [33]:
# Convert the December precipitation data from table to a list.
dec_prcp_list = [prcp.prcp for prcp in dec_prcp]
#print(june_prcp_list)

In [34]:
# Create a DataFrame from dec_prcp_list
dec_prcp_df = pd.DataFrame(dec_prcp_list, columns=["prcp"])

In [35]:
# Use Pandas to calcualte the summary statistics for the precipitation data
dec_prcp_df.describe()

,prcp
count,1405.000000
mean,0.216819
std,0.541399
min,0.000000
25%,0.000000
50%,0.030000
75%,0.150000
max,6.420000


## Query #5: Dynamic Query (Rain stats for user-specified dates)

In [36]:
# Write a function called `calc_prcp` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum rain for that range of dates
def calc_prcp(start_date, end_date):
    return session.query(func.min(Measurement.prcp), func.avg(Measurement.prcp), func.max(Measurement.prcp)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
print(calc_prcp('2012-02-28', '2012-03-05'))

[(0.0, 0.7043137254901962, 6.24)]
